In [7]:
import duckdb
import pandas as pd
from IPython.display import display

pd.set_option('display.max_columns', None)

In [8]:

# Path to DuckDB database
DB_PATH = '../../database/clinisys_all.duckdb'

use_schema = 'silver'

# List of relevant silver tables
silver_tables = [
    'view_micromanipulacao',
    'view_congelamentos_ovulos',
    'view_descongelamentos_ovulos',
    'view_congelamentos_embrioes',
    'view_embrioes_congelados',
    'view_descongelamentos_embrioes',
    'view_tratamentos',
    # 'view_micromanipulacao_oocitos',
]

# Table that needs special handling
micromanip_oocitos_table = 'view_micromanipulacao_oocitos'


# Prompt for prontuario
prontuario = '805418' #'171805'

# Connect to DuckDB
db = duckdb.connect(DB_PATH)

# Special handling for micromanipulacao_oocitos
print(f'\n--- Records from {use_schema}.{micromanip_oocitos_table} for prontuario={prontuario} (via view_micromanipulacao) ---')
try:
    # Get all codigo_ficha for this prontuario
    codigos = db.execute(f"""
        SELECT codigo_ficha FROM {use_schema}.view_micromanipulacao WHERE prontuario = ?
    """, [prontuario]).fetchall()
    codigo_list = [row[0] for row in codigos]
    if codigo_list:
        # Query oocitos table for all id_micromanipulacao in codigo_list
        placeholders = ','.join(['?']*len(codigo_list))
        query = f"""
            SELECT * FROM {use_schema}.{micromanip_oocitos_table}
            WHERE id_micromanipulacao IN ({placeholders})
            ORDER BY id_micromanipulacao, id
        """
        df = db.execute(query, codigo_list).df()
        if not df.empty:
            display(df)
        else:
            print('No records found.')
    else:
        print('No matching codigo_ficha found in view_micromanipulacao.')
except Exception as e:
    print(f'Error querying {use_schema}.{micromanip_oocitos_table}:', e)


for table in silver_tables:
    print(f'\n--- Records from {use_schema}.{table} for prontuario={prontuario} ---')
    # Some tables may not have prontuario, so try/except
    try:
        df = db.execute(f"""
            SELECT * FROM {use_schema}.{table}
            WHERE prontuario = ?
            order by 1
        """, [prontuario]).df()
        if not df.empty:
            print(f'Shape: {df.shape}')
            display(df)
        else:
            print('No records found.')
    except Exception as e:
        print(f'Error querying {use_schema}.{table}:', e)

db.close()


--- Records from silver.view_micromanipulacao_oocitos for prontuario=805418 (via view_micromanipulacao) ---


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,embryo_number
0,170486,17991,Não,MII,None,None,3888,None,Descartado,None,None,None,None,2CP,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1cab8ba3f8a107ab6e8fd7b10e76db26,2025-07-21 21:40:02,1
1,170487,17991,Não,MII,None,None,3888,3583,Criopreservado,None,Sim,Normal,PGT-A,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,8,1,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 1,None,None,Sim,None,Blastocisto Grau 5,B,A,None,None,None,None,None,None,None,ce0a194ec9e3b901e04206ffd8aa0985,2025-07-21 21:40:02,2
2,170488,17991,Não,MII,None,None,3888,3583,Criopreservado,None,Sim,Mosaico,PGT-A,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,8,2,Simétricos,None,None,None,None,None,None,Sim,None,Mórula A,None,None,Sim,None,Blastocisto Grau 1,None,None,Sim,None,Blastocisto Grau 6,B,B,None,None,628fd72df0dc770809420c60ec2774bb,2025-07-21 21:40:02,3
3,170489,17991,Não,MI,None,None,None,None,Descartado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6f760e7046d0ab7ec615ae5f13093b31,2025-07-21 21:40:02,4
4,170490,17991,Não,Gig.,None,None,None,None,Descartado,None,None,None,None,None,C0,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,b626a1a4f30ca114231f4c3f02c0aea8,2025-07-21 21:40:02,5



--- Records from silver.view_micromanipulacao for prontuario=805418 ---
Shape: (1, 211)


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,17991,2788/23,805418,35,38,2023-10-28,None,13:10,13:20,10:23,665,3888,None,Não,None,None,None,13:00,07:00,None,07:00,07:00,07:00,07:00,5,5,None,None,None,None,None,Não,None,1,1,None,2023-10-28,ICSI,03 espermatozoides móveis progressivos\r\n,03 espermatozoides normais\r\n,1,None,None,None,None,None,0000000002,None,None,None,3888,4004,None,3736,3583


--- Records from silver.view_congelamentos_ovulos for prontuario=805418 ---
No records found.

--- Records from silver.view_descongelamentos_ovulos for prontuario=805418 ---
No records found.

--- Records from silver.view_congelamentos_embrioes for prontuario=805418 ---
Shape: (1, 47)


...

,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,66832,170487,23692,14171,805418,#2,None,None,#2,None,None,None,Blastocisto Grau 5 - B - A,None,None,#2 EUPLOIDE,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Blastocisto Grau 5,B,A,None,None,None,None,None,None,None,None,None,None,Transferido,Sim,Normal,Euploide masculino,Sim,Normal,6.8,D6,None,None,None,None,None,None,3a7843c06243ad9090f1da5eeeba44f2,2025-07-21 21:38:57
1,66833,170488,23692,0,805418,#3,None,None,#3,None,None,None,Blastocisto Grau 6 - B - B,None,None,"Baixo grau XY, trissomia Chr 10 em mosaico (35%)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Descartado,Sim,Mosaico,"Baixo grau XY, trissomia Chr 10 em mosaico (35%)",Sim,Mosaico,3.2,D7,None,None,None,None,None,None,4cc1e5732d8a5b1f3d1d4346aaf356a3,2025-07-21 21:38:57



--- Records from silver.view_descongelamentos_embrioes for prontuario=805418 ---
Shape: (1, 34)


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Transferencia,DataTransferencia,Prateleira,Incubadora,transferidos_transferencia,cateter_transferencia,lote_transferencia,validade_transferencia,intercorrencia_transferencia,sangue_interno_transferencia,sangue_externo_transferencia,retorno_transferencia,vezes_retorno_transferencia,Transfer_D5,responsavel_transferencia,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,14171,421/24,2,805418,0,2023-11-03,2024-03-01,2788/23,M9293,E21357/23,A2,1,1,Desvitrificação,<NA>,2024-03-01,<NA>,4.2,1,GUARDIA,None,None,None,Não,Não,Não,None,None,NaN,None,None,None,e9498ffc30e001e56a41d2306855f933,2025-07-21 21:38:51



--- Records from silver.view_tratamentos for prontuario=805418 ---
Shape: (5, 193)


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,hora_procedimento,tipo_procedimento,motivo_congelamento,tipo_inducao,protocolo,tipo_ciclo_congelado,tipo_preparo_endometrial,num_congelados,num_transferir,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,material_heterologo,utero_substituicao,prontuario_genitores,prontuario_uter_substituicao,doacao_ovulos,destino_doacao_ovulos,prontuario_receptora,complemento_receptora,doacao_embrioes,era,emma,alice,matrice_lab,imunohistoquimica,outros_biopsia,consentimento,financeiro,embryoscope,pgt,era_auxiliar,recomendacao_transferencia,resultado_era,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,fator_infertilidade3,fsh_exame,foliculos_antrais,data_histeroscopia,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,ropa,data_inicio_agonista,gnrh,fsh_agonista,lh_agonista,outros_medicamentos,gonal,menogon,menopur,pergoveris,elonva,serofene_clomid_indux,luveris,puregon,letrozole,fostimon,merional,rekovelle,descricao_outras_medicacoes,dose_total_fsh,dose_total_lh,dose_total_fsh_lh,coorte_e2,coorte_aco,coorte_progesterona,coq10,hormonal_combinado,coorte_testosterona,agonista_coorte,coorte_antagonista,coorte_dhea,utilizado_hcg,motivo_cancelamento_hcg,data_hcg,horario_planejado_hcg,horario_real_hcg,ovidrel,gonapeptyl,dose_gonapeptyl,outras_complicacoes_hcg,dose_outras_complicacoes,lupron,outros_hcg,descricao_outros_hcg,data_inicio_progesterona,p4_inicio_progesterona,estradiol_fase_lutea,progesterona_fase_lutea,progesterona_oral,progesterona_vaginal,progesterona_injetavel,combinacao_oral_vaginal,nenhuma_progesterona,hcg_progesterona,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,observacoes_descongelamento,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,p4_transferencia,motivo_nao_transferir,motivo_cancelamento_tratamento,ligacao_enfermagem,beta_valor,beta_valor2,data_ultrassom1,data_ultrassom2,num_sacos_gestacionais,tsh,data_ultrassom1_contato,descricao_ultrassom1,bcf_embrião1_status,bcf_embrião1_fonte,data_ultrassom2_contato,descricao_ultrassom2,data_ultrassom3_contato,descricao_ultrassom3,resultado_tratamento,analise_genetica,resultado_analise_genetica,motivo_perda_aborto,data_nascimento_bebes,obstetra,nome_bebe1,sexo_bebe1,peso_bebe1,altura_bebe1,idade_gestacional_bebe1,via_parto_bebe1,viabilidade_bebe1,anomalias_bebe1,nome_bebe2,sexo_bebe2,peso_bebe2,altura_bebe2,idade_gestacional_bebe2,via_parto_bebe2,viabilidade_bebe2,anomalias_bebe2,nome_bebe3,sexo_bebe3,peso_bebe3,altura_bebe3,idade_gestacional_bebe3,via_parto_bebe3,viabilidade_bebe3,anomalias_bebe3,nome_bebe4,sexo_bebe4,peso_bebe4,altura_bebe4,idade_gestacional_bebe4,via_parto_bebe4,viabilidade_bebe4,anomalias_bebe4,observacoes_bebes,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,21279,805418,3,35,37,esposa,1,NaT,None,Ciclo a Fresco FIV,None,Convencional,Longo,None,Hormonal,None,None,Homólogo,Fresco,None,Homólogo,Sêmen,Fresco,None,None,Não,None,None,Não,None,None,None,Não,None,None,None,None,None,None,None,None,None,PGT-A,None,None,None,None,None,None,None,None,None,64,"1,66",Não,16/10/2023,None,None,None,Fator feminino anatômico,Inexplicado,Inexplicado,None,None,None,16/10/2023,None,esposa,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No transfer,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No transfer,Não,None,None,None,None,None,None